# OpenAI Function Calling Tutorial
Learn how to extend ChatGPT's capabilities with custom functions

## What is Function Calling?
- Allows ChatGPT to call external functions/APIs
- Converts natural language requests into structured function calls
- Enables ChatGPT to interact with real-world systems

## What we'll cover:
1. Basic function calling concepts
2. Function definition and schema
3. Simple examples (calculator, weather)
4. Advanced examples (database queries, API calls)
5. Best practices and error handling
6. Real-world applications

In [10]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [11]:
import openai

In [12]:
client = openai.OpenAI(api_key=openai.api_key)

In [13]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "What's 15 * 23 + 45?"}
    ]
)

print("Traditional Response:")
print(response.choices[0].message.content)

APIConnectionError: Connection error.

In [14]:
def get_open_ai_response(prompt):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": "You are specilized in GEN so anser the question like a GEN AI specilist"},
        {"role": "user", "content": prompt}
    ]
    )

    print("Traditional Response:")
    print(response.choices[0].message.content)


In [15]:
get_open_ai_response("Who is PM of India")

APIConnectionError: Connection error.

In [ ]:
get_open_ai_response("Who is PM of India")

Traditional Response:
As an AI specialized in general knowledge, I can confirm that as of now, the Prime Minister of India is Narendra Modi.


In [ ]:
get_open_ai_response("answer me 2+2*75")

Traditional Response:
The correct answer to the expression 2 + 2 * 75 is 152. Would you like to ask anything else?


In [ ]:
def get_open_ai_response(prompt):
    response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are an expert AI mathematician with deep knowledge across all areas of mathematics, from elementary arithmetic to advanced topics like topology, abstract algebra, real analysis, complex analysis, differential equations, number theory, combinatorics, probability theory, statistics, discrete mathematics, linear algebra, and mathematical logic."},
        {"role": "user", "content": prompt}
    ],
    )
    

    print("Traditional Response:")
    print(response.choices[0].message.content)


In [ ]:
get_open_ai_response("answer me 2+2*75")

Traditional Response:
According to the order of operations in mathematics (PEMDAS/BODMAS), you should perform multiplication before addition. So, you should first calculate \(2 \times 75\), and then add 2.

1. Multiply: \(2 \times 75 = 150\).
2. Add: \(2 + 150 = 152\).

The answer is 152.


In [ ]:
# Define a simple calculator function
def calculate(operation: str, a: float, b: float) -> float:
    """
    Perform basic mathematical operations
    
    Args:
        operation: The operation to perform (+, -, *, /)
        a: First number
        b: Second number
    
    Returns:
        Result of the calculation
    """
    if operation == "+":
        return a + b
    elif operation == "-":
        return a - b
    elif operation == "*":
        return a * b
    elif operation == "/":
        return a / b if b != 0 else "Error: Division by zero"
    else:
        return "Error: Unsupported operation"

# Test the function
result = calculate("*", 15, 23)
print(f"15 * 23 = {result}")

15 * 23 = 345


In [ ]:
import json
# Define the function schema for OpenAI
calculator_function = {
    "name": "calculate",
    "description": "Perform basic mathematical operations like addition, subtraction, multiplication, and division",
    "parameters": {
        "type": "object",
        "properties": {
            "operation": {
                "type": "string",
                "description": "The mathematical operation to perform",
                "enum": ["+", "-", "*", "/"]
            },
            "a": {
                "type": "number",
                "description": "The first number"
            },
            "b": {
                "type": "number",
                "description": "The second number"
            }
        },
        "required": ["operation", "a", "b"]
    }
}

print("📋 Function schema defined!")
print(json.dumps(calculator_function, indent=2))

📋 Function schema defined!
{
  "name": "calculate",
  "description": "Perform basic mathematical operations like addition, subtraction, multiplication, and division",
  "parameters": {
    "type": "object",
    "properties": {
      "operation": {
        "type": "string",
        "description": "The mathematical operation to perform",
        "enum": [
          "+",
          "-",
          "*",
          "/"
        ]
      },
      "a": {
        "type": "number",
        "description": "The first number"
      },
      "b": {
        "type": "number",
        "description": "The second number"
      }
    },
    "required": [
      "operation",
      "a",
      "b"
    ]
  }
}


#### Basic Function Calling Implementation

In [ ]:
def chat_with_calculator(user_message: str):
    """
    Chat with OpenAI using function calling for calculations
    """
    messages = [
        {"role": "system", "content": "Yo are calculation specilist."},
        {"role": "user", "content": user_message}
    ]
    
    # Make the initial request with function definition
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        functions=[calculator_function],
        function_call="auto"  # Let OpenAI decide when to call functions
    )
    
    message = response.choices[0].message
    if message.function_call:
        function_name = message.function_call.name
        function_args = json.loads(message.function_call.arguments)
        
        print(f"🔧 OpenAI wants to call: {function_name}")
        print(f"📝 Arguments: {function_args}")
        if function_name == "calculate":
            result = calculate(**function_args)

            messages.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": function_name,
                    "arguments": json.dumps(function_args)
                }
            })

            messages.append({
                "role": "function",
                "name": function_name,
                "content": str(result)
            })
            
            # Get final response from OpenAI
            final_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
        return final_response.choices[0].message.content



        



In [ ]:
# Test the function calling
result = chat_with_calculator("who is the PM of India")
print("🎯 Function Calling Result:")
print(result)

🎯 Function Calling Result:
None


In [ ]:
# Test the function calling
result = chat_with_calculator("What's 15 * 23 + 45? Please calculate step by step.")
print("🎯 Function Calling Result:")
print(result)

🔧 OpenAI wants to call: calculate
📝 Arguments: {'operation': '*', 'a': 15, 'b': 23}
🎯 Function Calling Result:
First, let's calculate 15 * 23 which equals 345. 

Now, let's add 345 to 45:

345 + 45 = 390

Therefore, 15 * 23 + 45 equals 390.


In [ ]:
result = chat_with_calculator("What's 15 * 23 + 45? Please calculate step by step.")


🔧 OpenAI wants to call: calculate
📝 Arguments: {'operation': '*', 'a': 15, 'b': 23}


In [ ]:
print(result)

The result of 15 * 23 is 345. Now, we will add 45 to this result. 

345 + 45 = 390

Therefore, 15 * 23 + 45 equals 390.


In [ ]:
# More complex example - Weather API
def get_weather(city: str, country: str = "US") -> str:
    """
    Get current weather for a city
    
    Args:
        city: Name of the city
        country: Country code (default: US)
    
    Returns:
        Weather information as string
    """
    # Using a mock weather API for demonstration
    # In real implementation, you'd use actual weather API
    mock_weather_data = {
        "new york": {"temp": 22, "condition": "sunny", "humidity": 60},
        "london": {"temp": 15, "condition": "cloudy", "humidity": 80},
        "tokyo": {"temp": 28, "condition": "rainy", "humidity": 75}
    }
    
    city_key = city.lower()
    if city_key in mock_weather_data:
        data = mock_weather_data[city_key]
        return f"Weather in {city}: {data['temp']}°C, {data['condition']}, humidity: {data['humidity']}%"
    else:
        return f"Weather data not available for {city}"

# Define weather function schema
weather_function = {
    "name": "get_weather",
    "description": "Get current weather information for a specific city",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Name of the city"
            },
            "country": {
                "type": "string",
                "description": "Country code (e.g., US, UK, JP)",
                "default": "US"
            }
        },
        "required": ["city"]
    }
}

print("🌤️ Weather function ready!")

🌤️ Weather function ready!


###  Multiple Functions Handler

In [ ]:
# Create a handler that can work with multiple functions
class FunctionHandler:
    def __init__(self):
        self.functions = {
            "calculate": calculate,
            "get_weather": get_weather
        }
        
        self.function_schemas = [
            calculator_function,
            weather_function
        ]
    
    def call_function(self, function_name: str, arguments: dict):
        """Call the appropriate function with given arguments"""
        if function_name in self.functions:
            return self.functions[function_name](**arguments)
        else:
            return f"Function {function_name} not found"
    
    def chat(self, user_message: str):
        """Enhanced chat with multiple function support"""
        messages = [{"role": "user", "content": user_message}]
        
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            functions=self.function_schemas,
            function_call="auto"
        )
        
        message = response.choices[0].message
        
        if message.function_call:
            function_name = message.function_call.name
            function_args = json.loads(message.function_call.arguments)
            
            print(f"🔧 Calling function: {function_name}")
            print(f"📝 Arguments: {function_args}")
            
            # Execute the function
            result = self.call_function(function_name, function_args)
            
            # Continue conversation with function result
            messages.extend([
                {
                    "role": "assistant",
                    "content": None,
                    "function_call": {
                        "name": function_name,
                        "arguments": json.dumps(function_args)
                    }
                },
                {
                    "role": "function",
                    "name": function_name,
                    "content": str(result)
                }
            ])
            
            final_response = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=messages
            )
            
            return final_response.choices[0].message.content
        
        return message.content

# Initialize the handler
handler = FunctionHandler()
print("🤖 Multi-function handler ready!")

🤖 Multi-function handler ready!


In [ ]:
handler.chat("what is the Temperature Tokyo")

🔧 Calling function: get_weather
📝 Arguments: {'city': 'Tokyo', 'country': 'JP'}


'The current temperature in Tokyo is 28°C.'

In [ ]:
handler.chat("what is the result of 2+2")

🔧 Calling function: calculate
📝 Arguments: {'operation': '+', 'a': 2, 'b': 2}


'The result of 2 + 2 is 4.'

In [ ]:

import openai

openai.api_key = ""

client = openai.OpenAI(api_key=openai.api_key)

response = client.responses.create(
  model="gpt-3.5-turbo",
  input=[
    {
      "role": "system",
      "content": [
        {
          "type": "input_text",
          "text": "You are a Socratic tutor. Use the following principles in responding to students:\n    \n    - Ask thought-provoking, open-ended questions that challenge students' preconceptions and encourage them to engage in deeper reflection and critical thinking.\n    - Facilitate open and respectful dialogue among students, creating an environment where diverse viewpoints are valued and students feel comfortable sharing their ideas.\n    - Actively listen to students' responses, paying careful attention to their underlying thought processes and making a genuine effort to understand their perspectives.\n    - Guide students in their exploration of topics by encouraging them to discover answers independently, rather than providing direct answers, to enhance their reasoning and analytical skills.\n    - Promote critical thinking by encouraging students to question assumptions, evaluate evidence, and consider alternative viewpoints in order to arrive at well-reasoned conclusions.\n    - Demonstrate humility by acknowledging your own limitations and uncertainties, modeling a growth mindset and exemplifying the value of lifelong learning."
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "input_text",
          "text": "Help me to understand the future of artificial intelligence."
        }
      ]
    }
  ],
  temperature=0.8,
  max_output_tokens=1024
)